In [79]:
# data_metadata_isolation.ipynb

# セル1: ライブラリのインポート
import pandas as pd
import ast

In [83]:
# セル2: CSVファイルの読み込み（パスは適宜変更）
df = pd.read_csv('../data/merged_output.csv')

C:\Users\taka2\AppData\Local\Temp\ipykernel_30188\2789071425.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/merged_output.csv')


In [84]:
# セル3: head_elements列のデータをパースして整理
def extract_head_info(head_elements_str):
    try:
        elements = ast.literal_eval(head_elements_str)
        tag_counts = {}
        title_text = ""
        meta_description = None
        og_title = None
        og_image = None
        script_srcs = []
        link_stylesheets = []

        for tag in elements:
            tag_name = tag.get("tag")
            attrs = tag.get("attributes", {})
            if tag_name:
                tag_counts[tag_name] = tag_counts.get(tag_name, 0) + 1
            if tag_name == "title":
                title_text = tag.get("text", "")
            if tag_name == "meta":
                if attrs.get("name") == "description":
                    meta_description = attrs.get("content", None)
                if attrs.get("property") == "og:title":
                    og_title = attrs.get("content", None)
                if attrs.get("property") == "og:image":
                    og_image = attrs.get("content", None)
            if tag_name == "script" and "src" in attrs:
                script_srcs.append(attrs["src"])
            if tag_name == "link" and attrs.get("rel") == "stylesheet":
                link_stylesheets.append(attrs.get("href", ""))
        
        return {
            "meta_count": tag_counts.get("meta", 0),
            "link_count": tag_counts.get("link", 0),
            "script_count": tag_counts.get("script", 0),
            "title_count": tag_counts.get("title", 0),
            "title_text": title_text,
            "meta_description": meta_description,
            "og_title": og_title,
            "og_image": og_image,
            "script_srcs": script_srcs,
            "link_stylesheets": link_stylesheets,
        }
    except Exception:
        return {
            "meta_count": 0,
            "link_count": 0,
            "script_count": 0,
            "title_count": 0,
            "title_text": "",
            "meta_description": None,
            "og_title": None,
            "og_image": None,
            "script_srcs": [],
            "link_stylesheets": [],
        }


In [85]:

# セル4: 整理結果をDataFrameに変換して結合
parsed_df = df["head_elements"].dropna().apply(extract_head_info).apply(pd.Series)
df = pd.concat([df, parsed_df], axis=1)


In [86]:
# セル5: 最初の数行を表示
df.head()

,Unnamed: 0,url,type,original_url,url_type,url_len,pri_domain,letters_count,digits_count,special_chars_count,shortened,is_domain_in_url,secure_http,have_ip,url_region,root_domain,original_url.1,final_url,redirect_chain,redirect_count,head_elements,timeout,has_meta_refresh,meta_refresh_url,duration_sec,meta_count,link_count,script_count,title_count,title_text,meta_description,og_title,og_image,script_srcs,link_stylesheets
0,0,br-icloud.com.br,phishing,br-icloud.com.br,2,16,br-icloud.com.br,13,0,3,0,0,0,0,Brazil,br-icloud,http://br-icloud.com.br,http://br-icloud.com.br/,NaN,0,NaN,False,False,NaN,4.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,mp3raid.com/music/krizz_kaliko.html,benign,mp3raid.com/music/krizz_kaliko.html,0,35,mp3raid.com,29,1,5,0,0,0,0,Global,mp3raid,http://mp3raid.com/music/krizz_kaliko.html,http://mp3raid.com/music/krizz_kaliko.html,NaN,0,NaN,False,False,NaN,2.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,bopsecrets.org/rexroth/cr/1.htm,benign,bopsecrets.org/rexroth/cr/1.htm,0,31,bopsecrets.org,25,1,5,0,0,0,0,Global,bopsecrets,http://bopsecrets.org/rexroth/cr/1.htm,https://bopsecrets.org/rexroth/cr/1.htm,"[{""url"": ""http://bopsecrets.org/rexroth/cr/1.h...",1,"[{'tag': 'meta', 'attributes': {'http-equiv': ...",False,False,NaN,6.41,7.0,1.0,0.0,1.0,Classics Revisited (1),,None,None,[],[../../css/rexroth.css]
3,3,http://garage-pirenne.be/index.php?option=com_...,defacement,http://www.garage-pirenne.be/index.php?option=...,1,77,garage-pirenne.be,60,7,17,0,1,0,0,Belgium,garage-pirenne,http://www.garage-pirenne.be/index.php?option=...,http://www.garage-pirenne.be/index.php?option=...,NaN,0,NaN,False,False,NaN,4.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,http://adventure-nicaragua.net/index.php?optio...,defacement,http://adventure-nicaragua.net/index.php?optio...,1,228,adventure-nicaragua.net,199,22,14,0,1,0,0,Global,adventure-nicaragua,http://adventure-nicaragua.net/index.php?optio...,http://adventure-nicaragua.net/index.php?optio...,NaN,0,NaN,False,True,http://adventure-nicaragua.net/index.php?optio...,4.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
# 文字列として空、もしくは NaN の行を削除
df = df[df['head_elements'].notna()]
df = df[df['head_elements'].str.strip() != '']
df = df[df['head_elements'].str.strip() != '[]']

# 結果を確認（件数とhead_elementsの一部を表示）
print(f"フィルタ後の件数: {len(df)}")
df['head_elements'].head()

フィルタ後の件数: 371357


2     [{'tag': 'meta', 'attributes': {'http-equiv': ...
6     [{'tag': 'meta', 'attributes': {'charset': 'ut...
7     [{'tag': 'meta', 'attributes': {'charset': 'ut...
9     [{'tag': 'title', 'attributes': {}, 'text': 'C...
10    [{'tag': 'link', 'attributes': {'rel': 'precon...
Name: head_elements, dtype: object

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 371357 entries, 2 to 617124
Data columns (total 35 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           371357 non-null  int64  
 1   url                  371357 non-null  object 
 2   type                 371357 non-null  object 
 3   original_url         371357 non-null  object 
 4   url_type             371357 non-null  int64  
 5   url_len              371357 non-null  int64  
 6   pri_domain           371357 non-null  object 
 7   letters_count        371357 non-null  int64  
 8   digits_count         371357 non-null  int64  
 9   special_chars_count  371357 non-null  int64  
 10  shortened            371357 non-null  int64  
 11  is_domain_in_url     371357 non-null  int64  
 12  secure_http          371357 non-null  int64  
 13  have_ip              371357 non-null  object 
 14  url_region           371357 non-null  object 
 15  root_domain          3

In [91]:
df = df.drop(columns=['Unnamed: 0', 'original_url.1','script_srcs'])

In [92]:
df.to_csv("../data/metadata_preprocessed.csv", index=False)